In [1]:
# IMPORT PACKAGES
from astropy.io import fits # FITS I/O
import ctypes # C/C++/CUDA shared libraries
import numpy as np # NumPy
import time # Timing

In [2]:
# SHARED LIBRARIES
clib = ctypes.CDLL("../src/Stacking/stack.o")

In [3]:
# DATA INGESTION
biasFrames = fits.open("../assets/NGC-1499/Bias/Bias.fit")
darkFrames = fits.open("../assets/NGC-1499/Dark/Dark.fit")

In [4]:
# DATA TRANSFORMATION

def flattenSubframes(subframes: list) -> list:
	"""
	Flattens a list of subframes.
	
	Args:
		subframes (list): A list of Astropy FITS HDU objects which contain the 
		individual subframes.
	Returns:
		A list of image statistics and NumPy ndarrays. The list contains, in 
		order, the following: number of subframes, image width, image height, 
		flattened and appended red color channels from each subframe, 
		flattened and appended green color channels from each subframe, 
		flattened and appended blue color channels from each subframe
	"""

	# Determine how much memory to allocate
	imageWidth = subframes[0].data.T.shape[0]
	imageHeight = subframes[0].data.T.shape[1]
	pixelsPerImage = imageWidth * imageHeight
	numberOfSubframes = len(subframes)
	dataType = subframes[0].data.dtype
	print("Image width:", subframes[0].data.T.shape[0])
	print("Image height:", subframes[0].data.T.shape[1])
	print("Pixels per image:", pixelsPerImage)
	print("Number of subframes:", numberOfSubframes)
	print("Data type:", dataType)
	
	# Allocate empty arrays
	flattenedRedChannels = np.empty((1, pixelsPerImage * numberOfSubframes), dtype = dataType)
	flattenedGreenChannels = np.empty((1, pixelsPerImage * numberOfSubframes), dtype = dataType)
	flattenedBlueChannels = np.empty((1, pixelsPerImage * numberOfSubframes), dtype = dataType)

	# Flatten subframes by color channel
	for subframe in range(0, len(subframes)):
		startIndex = subframe * pixelsPerImage
		endIndex = (subframe + 1) * pixelsPerImage

		flattenedRedChannels[0, startIndex:endIndex] = subframes[subframe].data.T[:, :, 0].flatten()
		flattenedGreenChannels[0, startIndex:endIndex] = subframes[subframe].data.T[:, :, 1].flatten()
		flattenedBlueChannels[0, startIndex:endIndex] = subframes[subframe].data.T[:, :, 2].flatten()

	return [numberOfSubframes, imageWidth, imageHeight, flattenedRedChannels.flatten(), flattenedGreenChannels.flatten(), flattenedBlueChannels.flatten()]

In [5]:
flattenedBiasFrames = flattenSubframes(biasFrames)

Image width: 4948
Image height: 3280
Pixels per image: 16229440
Number of subframes: 5
Data type: uint16


In [15]:
class Stack(ctypes.Structure):
    _fields_ = [
        ("numberOfSubframes", ctypes.c_uint64),
        ("imageWidth", ctypes.c_uint64),
        ("imageHeight", ctypes.c_uint64),
        ("pixelsPerImage", ctypes.c_uint64),
        ("redSubframes", ctypes.POINTER(ctypes.c_uint16)),
        ("greenSubframes", ctypes.POINTER(ctypes.c_uint16)),
        ("blueSubframes", ctypes.POINTER(ctypes.c_uint16)),
        ("stackedRed", ctypes.POINTER(ctypes.c_float)),
        ("stackedGreen", ctypes.POINTER(ctypes.c_float)),
        ("stackedBlue", ctypes.POINTER(ctypes.c_float))
        # Add more fields matching the C struct if needed
    ]

In [18]:
clib.initializeStack.restype = ctypes.POINTER(Stack)

red_data = np.array(flattenedBiasFrames[3], dtype=np.uint16)
green_data = np.array(flattenedBiasFrames[4], dtype=np.uint16)
blue_data = np.array(flattenedBiasFrames[5], dtype=np.uint16)

# Prepare ctypes data types
c_uint64 = ctypes.c_uint64
c_uint16_p = ctypes.POINTER(ctypes.c_uint16)

# Convert NumPy arrays to ctypes pointers
red_ptr = red_data.ctypes.data_as(c_uint16_p)
green_ptr = green_data.ctypes.data_as(c_uint16_p)
blue_ptr = blue_data.ctypes.data_as(c_uint16_p)

# Call the C function
x = clib.initializeStack(c_uint64(flattenedBiasFrames[0]),
                         c_uint64(flattenedBiasFrames[1]),
                         c_uint64(flattenedBiasFrames[2]),
                         red_ptr,
                         green_ptr,
                         blue_ptr)

0.0

: 